In [2]:
import threading
import os

import time
import datetime
from dateutil.relativedelta import relativedelta

import pandas as pd
import numpy as np

from libs.reading_utils import get_keywords, get_proxies, get_custom_keywords
from libs.writing_utils import get_logger, get_locations
from processor import historicProcessor, profileProcessor

from twitterscraper import query_historic_tweets
from livescraper import query_live_tweets

import tweepy

In [3]:
def download_live(keywords, logger):
    qt = query_live_tweets(keywords)
    listener, auth = qt.get_listener(create=True)

    while True:
        try:
            qt.perform_search()
        except KeyboardInterrupt:
            logger.warning("Keyboard interrupted. Saving whatever has been collected")
            df, userData, start_time = listener.get_data()
            t1 = threading.Thread(target=qt.save_data, args=[df, userData, start_time, int(time.time())])
            t1.start()
            break
        except Exception as e:
            logger.warning(('Got an exception. Error is: {}. Saving whatever exists').format(str(e)))    
            df, userData, start_time = listener.get_data()
            t1 = threading.Thread(target=qt.save_data, args=[df, userData, start_time, int(time.time())])
            t1.start()

liveKeywords, historicList = get_keywords()
proxies = get_proxies()

_, currRoot_dir = get_locations() 
logger = get_logger(currRoot_dir + '/logs/run_live.txt')

In [4]:
auth = tweepy.OAuthHandler("95fyXonGGIHKgfothfbOOAM7p", "6KWDuC87go4CbFE6jLdRnHWGFcj2Fl9hQvdizfaiwCOdZliv49")
auth.set_access_token("852009551876431872-OfvYX17eqrPz9eERGaRVxKfkBPVALyO", "koQa3hgW22EsgdvseQVsj3KnYbzHc564xEVfr7lYiPGhy")

api = tweepy.API(auth)

In [13]:
res = api.search(q="Bitcoin", since="2018-10-28", until="2018-10-29", count=1000, result_type='popular')

In [14]:
# In 15 minutes you can send 180 requests at max
for data in res:
    print(data)

Status(_api=<tweepy.api.API object at 0x7f793165ed68>, _json={'created_at': 'Sun Oct 28 14:08:01 +0000 2018', 'id': 1056548141313404929, 'id_str': '1056548141313404929', 'text': 'Bitcoin’s inflation schedule is set for the next 100 years: transparent, unwavering, and known to all parties. US D… https://t.co/qkrICVTXhC', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/qkrICVTXhC', 'expanded_url': 'https://twitter.com/i/web/status/1056548141313404929', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]}, 'metadata': {'result_type': 'popular', 'iso_language_code': 'en'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 61417559, 'id_str': '61417559', 'name': 'Erik Voorhees', 'screen_na

In [12]:
len(res)

0